<a href="https://colab.research.google.com/github/Tomawock/MLDM_COVID-19/blob/master/testing/covid_random_forest_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing per le Random Forest

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import joblib
from datetime import datetime

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.metrics import r2_score, mean_squared_error, max_error
from sklearn.ensemble import RandomForestRegressor 

from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

**Nomenclatura per i test**

In [2]:
path='/content/'
datasets_names= ['dataset_finale_medie_rnd1.csv','dataset_finale_medie_2_week_rnd1.csv','dataset_finale_medie_ponderate_rnd1.csv',
               'dataset_finale_medie_rnd220.csv','dataset_finale_medie_2_week_rnd220.csv','dataset_finale_medie_ponderate_rnd220.csv']

rdn_states= [1, 22, 777, 6654, 432145]
features_prediction= ['fut1_nuovi_positivi','fut2_nuovi_positivi',
                     'fut1_ricoverati_con_sintomi','fut2_ricoverati_con_sintomi',
                     'fut1_terapia_intensiva','fut2_terapia_intensiva',
                     'fut1_deceduti','fut2_deceduti',
                     'fut1_totale_ospedalizzati','fut2_totale_ospedalizzati',
                     'fut1_dimessi_guariti','fut2_dimessi_guariti']

coefficents=['R2','RMSE','MAXERR']

**Creazione indici per il dataset comune nel quale verranno messi i dati dei test**


In [3]:
col=[]
inde=[]
for feature in features_prediction:
  for coeff in coefficents:
    col.append(coeff+'_'+feature)
    #print(f"Columns_names\t{coeff+'_'+feature}")

for rnd in rdn_states:
  for path in datasets_names:
    inde.append(path+':'+str(rnd))
    #print(f"indexes_\t{path+':'+str(rnd)}")

df_results=pd.DataFrame(columns=col,index=inde)
print(df_results.columns)
print(df_results.index)

Index(['R2_fut1_nuovi_positivi', 'RMSE_fut1_nuovi_positivi',
       'MAXERR_fut1_nuovi_positivi', 'R2_fut2_nuovi_positivi',
       'RMSE_fut2_nuovi_positivi', 'MAXERR_fut2_nuovi_positivi',
       'R2_fut1_ricoverati_con_sintomi', 'RMSE_fut1_ricoverati_con_sintomi',
       'MAXERR_fut1_ricoverati_con_sintomi', 'R2_fut2_ricoverati_con_sintomi',
       'RMSE_fut2_ricoverati_con_sintomi',
       'MAXERR_fut2_ricoverati_con_sintomi', 'R2_fut1_terapia_intensiva',
       'RMSE_fut1_terapia_intensiva', 'MAXERR_fut1_terapia_intensiva',
       'R2_fut2_terapia_intensiva', 'RMSE_fut2_terapia_intensiva',
       'MAXERR_fut2_terapia_intensiva', 'R2_fut1_deceduti',
       'RMSE_fut1_deceduti', 'MAXERR_fut1_deceduti', 'R2_fut2_deceduti',
       'RMSE_fut2_deceduti', 'MAXERR_fut2_deceduti',
       'R2_fut1_totale_ospedalizzati', 'RMSE_fut1_totale_ospedalizzati',
       'MAXERR_fut1_totale_ospedalizzati', 'R2_fut2_totale_ospedalizzati',
       'RMSE_fut2_totale_ospedalizzati', 'MAXERR_fut2_totale_osped

In [4]:
# datasets_names= ['dataset_finale_medie_rnd1.csv','dataset_finale_medie_2_week_rnd1.csv']
# rdn_states= [1,22]
# features_prediction= ['fut1_nuovi_positivi','fut2_nuovi_positivi']


In [5]:
for rnd in rdn_states:
  for dataset in datasets_names:
    for feature in features_prediction:

      df_2 = pd.read_csv(dataset, parse_dates=["Data"], infer_datetime_format=True)
      rnd_state = rnd
      predictor_columns = [
            'pass_Ammoniaca', 
            'pass_Benzene',
            'pass_Biossido di Azoto', 
            'pass_Biossido di Zolfo',
            'pass_Monossido di Azoto', 
            'pass_Monossido di Carbonio',
            'pass_Ossidi di Azoto', 
            'pass_Ozono', 
            'pass_PM10 (SM2005)',
            'pass_Particelle sospese PM2.5',
            'pass_Radiazione Globale', 
            'pass_Temperatura',
            'pass_deceduti',
            'pass_nuovi_positivi',
            'pass_ricoverati_con_sintomi', 
            'pass_tamponi']
      ################################################################################
      train_2 = df_2.sample(frac=0.70, random_state=rnd_state)
      test_2 = df_2.drop(train_2.index)

      train_X_2 = train_2[predictor_columns]
      train_y_2 = train_2[feature]

      test_X_2 = test_2[predictor_columns]
      test_y_2 = test_2[feature]
      ################################################################################
      forest_regressor = RandomForestRegressor(bootstrap=False,random_state=rnd_state) 
      ################################################################################
      imp=SimpleImputer(missing_values=np.nan, strategy="mean")
      imp=imp.fit(train_X_2)
      ################################################################################
      n_estimators_list = [100]
      criterion_list = ["mse", "mae"]
      max_depth_list = [None, 5, 6]
      min_samples_split_list = [2, 3]
      min_samples_split_leaf_list = [3,5]
      min_weight_fraction_leaf_list = [0.0, 0.1]
      max_features_list = ["auto", "sqrt", "log2"]
      max_leaf_nodes_list = [None, 10, 25]
      min_impurity_decrease_list = [0.0, 0.1]

      param_grd = { "n_estimators":n_estimators_list,
                    "criterion": criterion_list,
                    "max_depth": max_depth_list,
                    "min_samples_split": min_samples_split_list,
                    "min_samples_leaf": min_samples_split_leaf_list,
                    "min_weight_fraction_leaf": min_weight_fraction_leaf_list,
                    "max_features": max_features_list,
                    "max_leaf_nodes": max_leaf_nodes_list,
                    "min_impurity_decrease": min_impurity_decrease_list}
      ################################################################################
      grid_regressor_2 = GridSearchCV(forest_regressor, param_grd, 
                                    n_jobs=-1, 
                                    verbose=0)
      ################################################################################
      grid_regressor_2.fit(imp.transform(train_X_2), train_y_2)
      ################################################################################
      best_regressor_2 = grid_regressor_2.best_estimator_

      imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
      imputer = imputer.fit(test_X_2)
      test_X_imp = imputer.transform(test_X_2)

      predicted_2 = best_regressor_2.predict(test_X_imp)
      ################################################################################
      r2=float("{:.3f}".format(r2_score(test_y_2, predicted_2)))
      rmse=float("{:.3f}".format(mean_squared_error(test_y_2, predicted_2, squared=False)))
      maxerr=float("{:.3f}".format(max_error(test_y_2, predicted_2)))
      print(f"R2: \t\t{r2}")
      print(f"RMSE: \t\t{rmse}")
      print(f"MAX ERR: \t{maxerr}")
      ################################################################################
      df_results.loc[[dataset+':'+str(rnd_state)], [coefficents[0]+'_'+feature]] = r2
      df_results.loc[[dataset+':'+str(rnd_state)], [coefficents[1]+'_'+feature]] = rmse
      df_results.loc[[dataset+':'+str(rnd_state)], [coefficents[2]+'_'+feature]] = maxerr

R2: 		0.869
RMSE: 		244.9
MAX ERR: 	958.5
R2: 		0.772
RMSE: 		366.592
MAX ERR: 	1704.065
R2: 		0.875
RMSE: 		238.521
MAX ERR: 	916.945
R2: 		0.814
RMSE: 		331.225
MAX ERR: 	1661.531
R2: 		0.853
RMSE: 		238.19
MAX ERR: 	878.751
R2: 		0.783
RMSE: 		387.051
MAX ERR: 	1697.78
R2: 		0.838
RMSE: 		250.48
MAX ERR: 	837.0
R2: 		0.82
RMSE: 		352.26
MAX ERR: 	1670.805


In [6]:
df_results.head(10)

,R2_fut1_nuovi_positivi,RMSE_fut1_nuovi_positivi,MAXERR_fut1_nuovi_positivi,R2_fut2_nuovi_positivi,RMSE_fut2_nuovi_positivi,MAXERR_fut2_nuovi_positivi,R2_fut1_ricoverati_con_sintomi,RMSE_fut1_ricoverati_con_sintomi,MAXERR_fut1_ricoverati_con_sintomi,R2_fut2_ricoverati_con_sintomi,RMSE_fut2_ricoverati_con_sintomi,MAXERR_fut2_ricoverati_con_sintomi,R2_fut1_terapia_intensiva,RMSE_fut1_terapia_intensiva,MAXERR_fut1_terapia_intensiva,R2_fut2_terapia_intensiva,RMSE_fut2_terapia_intensiva,MAXERR_fut2_terapia_intensiva,R2_fut1_deceduti,RMSE_fut1_deceduti,MAXERR_fut1_deceduti,R2_fut2_deceduti,RMSE_fut2_deceduti,MAXERR_fut2_deceduti,R2_fut1_totale_ospedalizzati,RMSE_fut1_totale_ospedalizzati,MAXERR_fut1_totale_ospedalizzati,R2_fut2_totale_ospedalizzati,RMSE_fut2_totale_ospedalizzati,MAXERR_fut2_totale_ospedalizzati,R2_fut1_dimessi_guariti,RMSE_fut1_dimessi_guariti,MAXERR_fut1_dimessi_guariti,R2_fut2_dimessi_guariti,RMSE_fut2_dimessi_guariti,MAXERR_fut2_dimessi_guariti
dataset_finale_medie_rnd1.csv:1,0.869,244.9,958.5,0.772,366.592,1704.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dataset_finale_medie_2_week_rnd1.csv:1,0.875,238.521,916.945,0.814,331.225,1661.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dataset_finale_medie_ponderate_rnd1.csv:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dataset_finale_medie_rnd220.csv:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dataset_finale_medie_2_week_rnd220.csv:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dataset_finale_medie_ponderate_rnd220.csv:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dataset_finale_medie_rnd1.csv:22,0.853,238.19,878.751,0.783,387.051,1697.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dataset_finale_medie_2_week_rnd1.csv:22,0.838,250.48,837,0.82,352.26,1670.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dataset_finale_medie_ponderate_rnd1.csv:22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dataset_finale_medie_rnd220.csv:22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_results.to_csv('covid_random_forest_test.csv')

In [8]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)
import os
os.chdir("/content/drive/Shared drives/MLDM progetto/Datasets/dataset_risultati_test")

Drive not mounted, so nothing to flush and unmount.


KeyboardInterrupt: ignored

In [ ]:
df_results.to_csv('covid_random_forest_test.csv')